In [ ]:
import requests
from io import BytesIO
import zipfile
import pandas as pd
import os

def download_and_extract_zip(zip_url, local_path):
    # Create the local directory if it doesn't exist
    os.makedirs(local_path, exist_ok=True)

    # Download the zip file
    response = requests.get(zip_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Extract the contents of the zip file
        with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
            zip_file.extractall(local_path)
            
            # Get the list of extracted files
            extracted_files = zip_file.namelist()
            
            # Assume the first file is the CSV file (modify as needed)
            csv_file = extracted_files[0]
            
            print(f"Zip file downloaded and extracted to {local_path}")
            
            return csv_file
    else:
        print(f"Failed to download the zip file. Status code: {response.status_code}")
        return None

def load_csv_to_dataframe(csv_path):
    df = pd.read_csv(csv_path)
    return df

zip_file_url = 'https://skinfix-ml-bucket.s3.amazonaws.com/hmnist_28_28_RGB.csv.zip'

# Specify the local directory for downloading and extracting
local_directory = './downloaded_data'  # Change this path as needed

# Download and extract the zip file, get the CSV file name
csv_file_name = download_and_extract_zip(zip_file_url, local_directory)

# if csv_file_name:
# Construct the full path to the CSV file
csv_file_path = os.path.join(local_directory, csv_file_name)

# Load the CSV file into a Pandas DataFrame
df = load_csv_to_dataframe(csv_file_path)

In [22]:
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import RMSprop

model=Sequential()

model.add(Conv2D(64,(2,2),input_shape=(28,28,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128,(2,2),input_shape=(28,28,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Dropout(0.3))

model.add(Conv2D(256,(2,2),input_shape=(28,28,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Dropout(0.3))

model.add(Conv2D(128,(1,1),input_shape=(28,28,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(BatchNormalization())
#
model.add(Dropout(0.3))
model.add(Conv2D(64,(1,1),input_shape=(28,28,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(BatchNormalization())

#
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 27, 27, 64)        832       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 13, 13, 64)        0         
 ng2D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 13, 13, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_16 (Conv2D)          (None, 12, 12, 128)       32896     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 6, 6, 128)         0         
 ng2D)                                                           
                                                      

In [24]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('downloaded_data/hmnist_28_28_RGB.csv')
# df=pd.read_csv(os.path.join(base,'hmnist_28_28_RGB.csv'))
x=df.drop('label',axis=1)
y=df['label']
x=x.to_numpy()
x=x/255
y=to_categorical(y)

In [52]:
x=x.reshape(-1,28,28,3)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [53]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8012, 28, 28, 3), (2003, 28, 28, 3), (8012, 7), (2003, 7))

In [54]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [55]:
#Training our CNN
class_weights = {0:1,1:0.5,2:1,3:1,4:1,5:1,6:1}
model.fit(X_train, y_train, epochs=50, batch_size=128, class_weight=class_weights, validation_data=(X_val, y_val))

Epoch 1/50
57/57 [==============================] - 14s 242ms/step - loss: 0.8841 - accuracy: 0.6713 - val_loss: 2.6250 - val_accuracy: 0.0287
Epoch 2/50
57/57 [==============================] - 16s 278ms/step - loss: 0.8127 - accuracy: 0.6854 - val_loss: 2.8725 - val_accuracy: 0.0287
Epoch 3/50
57/57 [==============================] - 14s 242ms/step - loss: 0.7622 - accuracy: 0.7035 - val_loss: 3.2138 - val_accuracy: 0.0287
Epoch 4/50
57/57 [==============================] - 14s 238ms/step - loss: 0.7353 - accuracy: 0.7114 - val_loss: 3.3066 - val_accuracy: 0.0374
Epoch 5/50
57/57 [==============================] - 14s 241ms/step - loss: 0.7185 - accuracy: 0.7141 - val_loss: 2.9574 - val_accuracy: 0.1097
Epoch 6/50
57/57 [==============================] - 14s 237ms/step - loss: 0.6928 - accuracy: 0.7244 - val_loss: 2.7884 - val_accuracy: 0.0948
Epoch 7/50
57/57 [==============================] - 13s 234ms/step - loss: 0.6678 - accuracy: 0.7262 - val_loss: 2.4712 - val_accuracy: 0.1160

In [56]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

26/26 [==============================] - 0s 12ms/step - loss: 0.7462 - accuracy: 0.7656
Validation: accuracy = 0.765586  ;  loss_v = 0.746213
Test: accuracy = 0.746380  ;  loss = 0.749248


In [28]:
model.predict(X_val[:2])

1/1 [==============================] - 0s 158ms/step


array([[0.22128604, 0.19976686, 0.22389811, 0.09912382, 0.13297969,
        0.06667622, 0.05626924],
       [0.21781537, 0.20111354, 0.22002752, 0.09848139, 0.13805744,
        0.06696871, 0.05753605]], dtype=float32)

In [57]:
!rm -rf export

model.save('export/Servo/1', save_format='tf')

INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


In [58]:
!rm model.tar.gz

import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [59]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='skinfix-model-v10')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [15]:
!touch train.py

In [60]:
import boto3, re
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/skinfix-model-v10/model.tar.gz',
                                  role = role,
                                  framework_version = '2.13',
                                  entry_point = 'train.py')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [61]:
# Deploy a SageMaker to an endpoint
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium',
                                   endpoint_name='skinfix-ml-endpoint-v10')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
-----!

In [62]:
# Create a predictor which uses this new endpoint
import numpy as np

import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()

endpoint = 'skinfix-ml-endpoint-v10'

predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint, sagemaker_session)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [63]:
# .predict send the data to our endpoint
predictor.predict(X_val[:2])['predictions']

[[2.57025873e-10,
  1.14217976e-07,
  6.27406553e-06,
  1.92746242e-07,
  0.999984622,
  1.9768504e-06,
  6.8022614e-06],
 [0.745594084,
  0.185829148,
  0.0062421062,
  0.0518042631,
  0.00857963134,
  3.21644438e-05,
  0.00191857188]]

In [64]:
model.predict(X_val[:2])

1/1 [==============================] - 0s 27ms/step


array([[2.57025873e-10, 1.14217976e-07, 6.27407144e-06, 1.92746612e-07,
        9.99984622e-01, 1.97685222e-06, 6.80226776e-06],
       [7.45594203e-01, 1.85829103e-01, 6.24209829e-03, 5.18042184e-02,
        8.57962482e-03, 3.21643893e-05, 1.91856944e-03]], dtype=float32)

In [65]:
differences = np.abs(predictor.predict(X_val[:2])['predictions'] - model.predict(X_val[:2]))
differences

1/1 [==============================] - 0s 20ms/step


array([[1.92596236e-19, 4.77975941e-16, 5.90838354e-12, 3.69560324e-13,
        1.64790404e-12, 1.81792199e-12, 6.36239579e-12],
       [1.19472137e-07, 4.50069885e-08, 7.91325207e-09, 4.46587521e-08,
        6.52322468e-09, 5.45324362e-11, 2.43997789e-09]])

In [66]:
np.allclose(differences, np.zeros_like(differences), atol=1e-6)

True

In [51]:
import json

json.dump(X_val[:2].tolist(), open('test_data.txt', 'w+'),)